# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [84]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk import word_tokenize, pos_tag, ne_chunk, WordNetLemmatizer
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
# import category_encoders as ce

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/Disaster_Response.db')
# df = pd.read_sql("SELECT * FROM Disaster_Response", engine) # alternative
df = pd.read_sql_table("Disaster_Response", engine)
X = df.loc[:,'message':'genre']
Y = df.loc[:,'related':]
X.shape, Y.shape

((26180, 3), (26180, 36))

### 2. Write a tokenization function to process your text data

In [3]:
# regex pattern for detecting a url
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# define tokenizer
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex,text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# test the tokenizer
text=X.original[0]
print(tokenize(text))

['un', 'front', 'froid', 'se', 'retrouve', 'sur', 'cuba', 'ce', 'matin', '.', 'il', 'pourrait', 'traverser', 'haiti', 'demain', '.', 'des', 'averses', 'de', 'pluie', 'isolee', 'sont', 'encore', 'prevue', 'sur', 'notre', 'region', 'ce', 'soi']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# simple 3-step pipeline for text-processing and classification
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X.message, Y, random_state=1234)

# train the pipeline
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [8]:
y_test.shape, y_pred.shape

((6545, 36), (6545, 36))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# show prediction results
for i, col in enumerate(Y.columns):
    print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(y_test.iloc[:,i], y_pred[:,i]))

related 
               precision    recall  f1-score   support

           0       0.56      0.44      0.49      1527
           1       0.84      0.89      0.86      4978
           2       0.42      0.28      0.33        40

    accuracy                           0.78      6545
   macro avg       0.61      0.54      0.56      6545
weighted avg       0.77      0.78      0.77      6545

request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      5465
           1       0.83      0.39      0.53      1080

    accuracy                           0.89      6545
   macro avg       0.86      0.69      0.73      6545
weighted avg       0.88      0.89      0.87      6545

offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6519
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6468
           1       0.00      0.00      0.00        77

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0.98      6545

other_infrastructure 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6265
           1       0.50      0.00      0.01       280

    accuracy                           0.96      6545
   macro avg       0.73      0.50      0.49      6545
weighted avg       0.94      

### 6. Improve your model
Use grid search to find better parameters. 

In [54]:
# for gird-search MultiOutputClassifier is used 
# for extending classifiers that do not natively support multi-target classification 

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=0)))
])

In [26]:
# peek at tunable parameters
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a2216a158>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',
           

In [58]:
# setup tunable parameters
parameters = {
#             'vect__ngram_range': ((1, 1), (1, 2)),
#             'vect__max_df': (0.5, 0.75, 1.0),
#             'vect__max_features': (None, 5000, 10000),
              'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [100, 200, 500],
#               'clf__estimator__min_samples_split': [2,4,8]
              }

In [59]:
%%time
# setup gridsearch
cv = GridSearchCV(pipeline, param_grid=parameters,return_train_score=True, verbose=5, n_jobs=-1) 
# fit the data
cv.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  18 | elapsed: 13.2min remaining: 10.6min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 25.2min finished


CPU times: user 12min 30s, sys: 1.9 s, total: 12min 32s
Wall time: 37min 43s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [60]:
cv.best_score_

0.2426279602750191

In [61]:
cv.best_params_

{'clf__estimator__n_estimators': 200, 'tfidf__use_idf': True}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [64]:
# predict and show the result
y_pred_tuned = cv.predict(X_test)

for i, col in enumerate(Y.columns):
    print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(y_test.iloc[:,i], y_pred_tuned[:,i]))

related 
               precision    recall  f1-score   support

           0       0.74      0.24      0.37      1527
           1       0.80      0.97      0.88      4978
           2       0.53      0.23      0.32        40

    accuracy                           0.80      6545
   macro avg       0.69      0.48      0.52      6545
weighted avg       0.79      0.80      0.76      6545

request 
               precision    recall  f1-score   support

           0       0.90      0.99      0.94      5465
           1       0.90      0.45      0.60      1080

    accuracy                           0.90      6545
   macro avg       0.90      0.72      0.77      6545
weighted avg       0.90      0.90      0.89      6545

offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


tools 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6505
           1       0.00      0.00      0.00        40

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      0.99      0.99      6545

hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.00      0.00      0.00        68

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0.98      6545

shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6519
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [96]:
# add categorical feature through the pipeline

preprocessor = ColumnTransformer(transformers=[
     ('text', TfidfVectorizer(stop_words = 'english', ngram_range=(1,3)), 'message'), # 'message' VS ['message'] why error??
     ('cate', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['genre'])
])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=500,n_jobs=-1))])

X_train, X_test, y_train, y_test = train_test_split(X[['message','genre']], Y, random_state = 1234)

clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  TfidfVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.float64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df=1.0,
                           

In [97]:
# predict and show the result
y_pred_multi_feat = clf.predict(X_test)

for i, col in enumerate(Y.columns):
    print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(y_test.iloc[:,i], y_pred_multi_feat[:,i]))

related 
               precision    recall  f1-score   support

           0       0.67      0.49      0.57      1527
           1       0.85      0.93      0.89      4978
           2       0.78      0.17      0.29        40

    accuracy                           0.82      6545
   macro avg       0.77      0.53      0.58      6545
weighted avg       0.81      0.82      0.81      6545

request 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      5465
           1       0.81      0.51      0.63      1080

    accuracy                           0.90      6545
   macro avg       0.86      0.74      0.78      6545
weighted avg       0.89      0.90      0.89      6545

offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6519
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6468
           1       0.00      0.00      0.00        77

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0.98      6545

other_infrastructure 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6265
           1       0.00      0.00      0.00       280

    accuracy                           0.96      6545
   macro avg       0.48      0.50      0.49      6545
weighted avg       0.92      

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.